In [ ]:
# Embedding weights: float embedding_weights[vocab_size][embed_dim]
# GRU weights: float gru_kernel[input_dim][units]
#              float gru_recurrent_kernel[units][units]
#              float gru_bias[3 * units]
# Dense weights: float dense_kernel[units][2]
#                float dense_bias[2]
# Tokenizer: const char* vocab[] = { "PAD", "turn", "on", "off", ... };

In [6]:
import tensorflow as tf
import numpy as np
import pickle

In [7]:
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # +1 for padding

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=8, input_length=5, name="embedding"),
    tf.keras.layers.GRU(16, name="gru"),
    tf.keras.layers.Dense(2, activation='softmax', name="dense")
])
model.build(input_shape=(None, 5))
model.load_weights("gru_led_intent_weights.weights.h5")

In [12]:
embedding_layer = model.get_layer("embedding")
gru_layer = model.get_layer("gru")
dense_layer = model.get_layer("dense")

embedding_dim = embedding_layer.output_dim
input_length = model.input_shape[1]
gru_units = gru_layer.units
output_units = dense_layer.units

In [14]:
embedding_weights = embedding_layer.get_weights()[0]
gru_kernel, gru_recurrent, gru_bias = gru_layer.get_weights()
dense_kernel, dense_bias = dense_layer.get_weights()

In [15]:
max_vocab = vocab_size
vocab_list = ['PAD'] * max_vocab
for word, idx in word_index.items():
    vocab_list[idx] = word

In [16]:
def array_to_c(name, arr):
    flat = arr.flatten()
    c_array = f"// shape: {arr.shape}\n"
    c_array += f"float {name}[{len(flat)}] = {{\n  "
    c_array += ', '.join(f"{x:.6f}" for x in flat)
    c_array += "\n};\n"
    return c_array

In [17]:
with open("model_weights.h", "w") as f:
    f.write("#ifndef MODEL_WEIGHTS_H\n#define MODEL_WEIGHTS_H\n\n")

    # === Constants ===
    f.write("// === Model Constants ===\n")
    f.write(f"#define VOCAB_SIZE {vocab_size}\n")
    f.write(f"#define EMBED_DIM {embedding_dim}\n")
    f.write(f"#define SEQ_LEN {input_length}\n")
    f.write(f"#define GRU_UNITS {gru_units}\n")
    f.write(f"#define OUTPUT_CLASSES {output_units}\n\n")

    # === Weights ===
    f.write("// === Embedding Weights ===\n")
    f.write(array_to_c("embedding_weights", embedding_weights))

    f.write("\n// === GRU Weights ===\n")
    f.write(array_to_c("gru_kernel", gru_kernel))
    f.write(array_to_c("gru_recurrent", gru_recurrent))
    f.write(array_to_c("gru_bias", gru_bias))

    f.write("\n// === Dense Weights ===\n")
    f.write(array_to_c("dense_kernel", dense_kernel))
    f.write(array_to_c("dense_bias", dense_bias))

    # === Vocab ===
    f.write("\n// === Tokenizer Vocabulary ===\n")
    f.write(f"const char* vocab[VOCAB_SIZE] = {{\n")
    f.write(",\n".join([f'  "{w}"' for w in vocab_list]))
    f.write("\n};\n")

    f.write("\n#endif // MODEL_WEIGHTS_H\n")